In [ ]:
import pickle

import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import f_oneway, shapiro
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from statsmodels.stats.multicomp import MultiComparison

## Prepração dos dados

In [ ]:
with open("cover_type.pkl", "rb") as f:
    X_cover_train, y_cover_train, X_cover_teste, y_cover_teste = pickle.load(f)

In [ ]:
X_cover_train.shape, y_cover_train.shape

In [ ]:
X_cover_teste.shape, y_cover_teste.shape

In [ ]:
X_cover = np.concatenate((X_cover_train, X_cover_teste), axis = 0)
y_cover = np.concatenate((y_cover_train, y_cover_teste), axis = 0)

In [ ]:
X_cover.shape, y_cover.shape

### Arvore de decisão

In [ ]:
parametros = {
    "criterion": ["gini", "entropy"],
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 5, 10],
    "max_features": ["sqrt", "log2"],
    "splitter": ["best", "random"],
}

In [ ]:
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid = parametros)
grid_search.fit(X_cover, y_cover)
best_params = grid_search.best_params_
best_result = grid_search.best_score_
print(best_params, best_result)

### Random forest

In [23]:
parametros = {
    "n_estimators": [100, 200, 300],  # número de árvores na floresta
    "criterion": ["gini", "entropy"],  # função de avaliação da qualidade da divisão
    "max_depth": [3, 5, 10, None],  # profundidade máxima da árvore
    "min_samples_split": [2, 5, 10, 20],  # mínimo de amostras para dividir um nó
    "min_samples_leaf": [1, 2, 5, 10],  # mínimo de amostras em uma folha
}

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid = parametros)
grid_search.fit(X_cover, y_cover)
best_params = grid_search.best_params_
best_result = grid_search.best_score_
print(best_params, best_result)

### kNN

In [ ]:
parametros = {
    "n_neighbors": [3, 5, 7, 9, 11],  # número de vizinhos a considerar
    "weights": ["uniform", "distance"],  # pesos dos vizinhos
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],  # algoritmo usado para computar os vizinhos
    "p": [1, 2],  # parâmetro para a métrica de distância (1 = Manhattan, 2 = Euclidiana)
    "leaf_size": [10, 20, 30, 40, 50]  # tamanho das folhas (afeta desempenho, não acurácia)
}

In [ ]:
grid_search = GridSearchCV(estimator=KNeighborsClassifier(), param_grid = parametros)
grid_search.fit(X_cover, y_cover)
best_params = grid_search.best_params_
best_result = grid_search.best_score_
print(best_params, best_result)

### Regressão Logistica

In [ ]:
parametros = {
    "penalty": ["l1", "l2", "elasticnet", None],  # tipo de regularização
    "C": [0.01, 0.1, 1, 10, 100],  # força da regularização (quanto menor, mais forte)
    "solver": ["liblinear", "saga", "lbfgs", "newton-cg"],  # algoritmo para otimização
    "max_iter": [100, 200, 500],  # número máximo de iterações
    "class_weight": [None, "balanced"],  # lida com classes desbalanceadas
    "l1_ratio": [0, 0.5, 1]  # usado apenas quando penalty="elasticnet"
}

In [ ]:
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid = parametros)
grid_search.fit(X_cover, y_cover)
best_params = grid_search.best_params_
best_result = grid_search.best_score_
print(best_params, best_result)

### SVM

In [ ]:
parametros = {
    "C": [0.1, 1, 10, 100],  # penalidade por erro (controle de overfitting)
    "kernel": ["linear", "poly", "rbf", "sigmoid"],  # tipo de função do kernel
    "degree": [2, 3, 4],  # grau do polinômio (usado apenas se kernel="poly")
    "gamma": ["scale", "auto", 0.01, 0.1, 1],  # parâmetro do kernel RBF, poly e sigmoid
    "coef0": [0.0, 0.1, 0.5, 1.0],  # coeficiente usado nos kernels poly e sigmoid
    "class_weight": [None, "balanced"],  # tratamento de classes desbalanceadas
    "shrinking": [True, False],  # se usa heurística de shrinking (pode melhorar performance)
    "probability": [False, True],  # se deve calibrar para estimar probabilidades (mais lento)
}

### Redes Neurais

In [ ]:
parametros = {
    "hidden_layer_sizes": [(50,), (100,), (100, 50), (50, 50, 50)],  # tamanho e número de camadas ocultas
    "activation": ["identity", "logistic", "tanh", "relu"],  # função de ativação
    "solver": ["lbfgs", "sgd", "adam"],  # algoritmo de otimização
    "alpha": [0.0001, 0.001, 0.01, 0.1],  # parâmetro de regularização L2
    "learning_rate": ["constant", "invscaling", "adaptive"],  # estratégia de ajuste da taxa de aprendizado
    "learning_rate_init": [0.001, 0.01, 0.1],  # taxa de aprendizado inicial
    "max_iter": [200, 300, 500, 1000, 2000],  # número máximo de iterações
    "early_stopping": [True, False],  # interrompe se não houver melhora no score
}

In [ ]:
grid_search = GridSearchCV(estimator=MLPClassifier(), param_grid = parametros)
grid_search.fit(X_cover, y_cover)
best_params = grid_search.best_params_
best_result = grid_search.best_score_
print(best_params, best_result)